In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

!pip install -q pydantic==2.5.3
!pip install -q langchain-core==0.1.52
!pip install -q langchain==0.1.17
!pip install -q langchain-community==0.0.37
!pip install -q langchain-nvidia-ai-endpoints==0.0.8
!pip install -q pymupdf
!pip install -q faiss-cpu
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 7.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.5 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.50 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
langchain 0.3.22 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 2.5.3 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 2.5.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's depen

In [27]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document


In [3]:
os.environ["NVIDIA_API_KEY"] = "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"


In [28]:
chat_llm = ChatNVIDIA(model="ai-gemma-2-9b-it")
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4")

pdf_path = "/kaggle/input/bookrag/Principles-of-Bloodstain-Patter-Analysis-1.pdf"
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""]
)
docs_split = text_splitter.split_documents(docs)
print("Split complete")

documents_to_embed = [
    Document(
        page_content=doc.page_content,
        metadata={
            "source": f"page_{doc.metadata.get('page', i)}",
            "page": doc.metadata.get("page", i)
        }
    )
    for i, doc in enumerate(docs_split)
]
print("Embedding documents prepared")

vectorstore = FAISS.from_documents(documents_to_embed, embedder)
vectorstore.save_local("faiss_index")
print("FAISS index created and saved successfully.")

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context below. Be specific and concise.\n"
    "<context>\n"
    "{context}\n"
    "</context>\n"
    "Question: {input}"
)

retrieval_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=chat_llm,
    retriever=retriever,
    chain_type="stuff",  
    return_source_documents=True
)

question = "What are the categories and subcategories in BPA?"
response = retrieval_chain.invoke({"question": question})

print("Answer:\n", response["answer"])
print("\nSources:")
for i, doc in enumerate(response["source_documents"]):
    print(f"\n--- Source {i+1} ---")
    print("Source:", doc.metadata.get("source", "N/A"))
    print("Page:", doc.metadata.get("page", "N/A"))
    print(doc.page_content[:500], "...\n")

Split complete
Embedding documents prepared
FAISS index created and saved successfully.
Answer:
 The primary bloodstain pattern categories are passive, spatter, and altered. 

Each of these categories has subcategories:

* **Passive:** Gravity, Saturation/Pooling, Free Falling
* **Spatter:** Impact, Projection
* **Altered:**  Diluted, Clotted, Dried, Diffused, Sequenced, Insects, Voids, Contact, Flow, Secondary



Sources:

--- Source 1 ---
Source: page_40
Page: 40
to begin with the primary bloodstain pattern categories — namely, passive, spatter, and
altered — and proceed to speciﬁc subcategories based on case-driven facts.
Figure 1.5
Bloodstain pattern categories.
Bloodstain Pattern Categories
Bloodstains
Passive
Spatter
Altered
Transfer
Flow
Secondary
Mechanism
Diluted
Impact
Mechanism
Clotted
Large
Volume
Drop(s)
Projection
Mechanism
Diffused
Sequenced
Insects
Voids ...


--- Source 2 ---
Source: page_99
Page: 99
levels your analysis may attain dependent on available facts. As you 